# Importation du fichier initial

In [ ]:
import numpy as np
import pandas as pd
import csv
import pickle

train_enz = pd.read_csv(r'C:\Users\leper\OneDrive\Documents\Cours et projet OC\Projet 8\train.csv', encoding ='utf8')
train_enz

In [ ]:
train_enz.info()

On peut ici voir, que nous avons 5 colonnes. L'une d'elle, data_source, ne nous sera pas d'utilité pour ce projet, nous pouvons donc la supprimer.

In [ ]:
train_enz=train_enz.drop(['data_source'],axis=1)

# Importation du fichier contenant les lignes modifiées

In [ ]:
train_enz_2 = pd.read_csv(r'C:\Users\leper\OneDrive\Documents\Cours et projet OC\Projet 8\train_updates_20220929.csv', encoding ='utf8')
train_enz_2

In [ ]:
train_enz_2.info()

# Remplacement des données modifiées dans le premier dataset

In [ ]:
train_enz_modif=train_enz_2.loc[(train_enz_2['protein_sequence'].notna())|(train_enz_2['pH'].notna())|(train_enz_2['tm'].notna())]

In [ ]:
train_enz_modif

Transformons la colonne seq id en index

In [ ]:
train_enz_modif.set_index('seq_id',inplace=True)

In [ ]:
list_modif=train_enz_modif.index.tolist()

In [ ]:
for i in list_modif:
    train_enz.at[i,'protein_sequence']=train_enz_modif.at[i,'protein_sequence']
    train_enz.at[i,'pH']=train_enz_modif.at[i,'pH']
    train_enz.at[i,'tm']=train_enz_modif.at[i,'tm']

# Observation des doublons

In [ ]:
train_enz.loc[train_enz[['protein_sequence','pH']].duplicated(keep=False),:]

In [ ]:
Nous voyons ici qu'il y a plusieurs doublons, cependant, il faut prendre en compte que ces proteines ont également pu être testé dans des dilutions diverses ce qui entraine cette différence de tm entre proteine de même séquence testées au même pH.

# Observation des valeurs abberantes 

## Colonne protein sequence

In [1]:
train_enz['protein_sequence'].describe()

NameError: name 'train_enz' is not defined

On voit ainsi, que nous avons 28981 séquences protéiniques différentes, dont une qui a été testée 14 fois.

## Colonne pH

In [ ]:
train_enz['pH'].describe()

On voit ici, que le nombre min est de 1.99 et le max est de 53.4. Hors on sait qu'un pH varie entre 0 et 14. Observons les valeurs au dessus de 14 de plus près.

In [ ]:
train_enz.loc[train_enz['pH']>14]

Remplaçons ces valeurs par des valeurs nulles.

In [ ]:
ph_abb=train_enz.loc[train_enz['pH']>14].index.tolist()
for i in ph_abb:
    train_enz.at[i,'pH']=np.nan
train_enz.loc[train_enz['pH']>14]   

## Colonne Température de fusion

In [ ]:
train_enz['tm'].describe()

On voit ainsi, que la valeur minimale est de -1 et la valeur maximale est de 130. Hors lors de nos recherches, nous avons trouvé que les valeurs de tm variait entre 25 et 113 °C.

In [ ]:
train_enz.loc[(train_enz['tm']>113)|(train_enz['tm']<25)]

# Valeurs manquantes

In [ ]:
listetaux=[]
for i in train_enz.columns:
    nb_null = train_enz[i].isnull().sum()
    nb_lignetot = len(train_enz[i])
    taux_remplissage = ((nb_lignetot-nb_null)/nb_lignetot)*100
    listetaux.append([i,taux_remplissage])

listetaux.sort(key=lambda x:x[1], reverse=True)

for i,taux_remplissage in listetaux :
    print("{:<40} -> {}%".format(i,round(taux_remplissage,2)))

In [ ]:
Contient_na=train_enz.loc[(train_enz['pH']==np.nan)|(train_enz['pH'].isna())|(train_enz['tm']==np.nan)|(train_enz['tm'].isna())]
(len(Contient_na)/len(train_enz))*100

Etant donné que moins de 4 pourcents des données sont manquantes, nous pouvons supprimer les lignes en question.

In [ ]:
liste_lignes_Na=Contient_na.index
train_enz=train_enz.drop(liste_lignes_Na)

In [ ]:
listetaux=[]
for i in train_enz.columns:
    nb_null = train_enz[i].isnull().sum()
    nb_lignetot = len(train_enz[i])
    taux_remplissage = ((nb_lignetot-nb_null)/nb_lignetot)*100
    listetaux.append([i,taux_remplissage])

listetaux.sort(key=lambda x:x[1], reverse=True)

for i,taux_remplissage in listetaux :
    print("{:<40} -> {}%".format(i,round(taux_remplissage,2)))

Notre Dataset de training est donc désormais nettoyé. Passons désormais au nettoyage de notre dataset de test.

# Importation du fichier test

In [ ]:
test_enz = pd.read_csv(r'C:\Users\leper\OneDrive\Documents\Cours et projet OC\Projet 8\test.csv', encoding ='utf8')

In [ ]:
test_enz.info()

Nous allons supprimer la colonne data_source de ce fichier.

In [ ]:
test_enz=test_enz.drop(['data_source'],axis=1)

## Colonne pH

In [ ]:
test_enz['pH'].describe()

Ici, il ne semble pas y avoir de valeur aberrante. Nous allons observer si il y a des valeurs manquantes.

In [ ]:
listetaux=[]
for i in test_enz.columns:
    nb_null = test_enz[i].isnull().sum()
    nb_lignetot = len(test_enz[i])
    taux_remplissage = ((nb_lignetot-nb_null)/nb_lignetot)*100
    listetaux.append([i,taux_remplissage])

listetaux.sort(key=lambda x:x[1], reverse=True)

for i,taux_remplissage in listetaux :
    print("{:<40} -> {}%".format(i,round(taux_remplissage,2)))

Il n'y a pas de valeurs manquantes ici. Nous allons donc pouvoir passer à la suite.